In [ ]:
sc.install_pypi_package("pybind11==2.10.3")
sc.install_pypi_package("numpy==1.19.0")
sc.install_pypi_package("Pillow==8.2")
sc.install_pypi_package("Cython==0.29.33")
sc.install_pypi_package("scipy==1.2.0")
sc.install_pypi_package("pythran==0.12.1")
sc.install_pypi_package("pandas==1.0.0")
sc.install_pypi_package("matplotlib==3.3.0")
sc.install_pypi_package("lifelines==0.27.4")
sc.install_pypi_package("s3fs==0.4.2")
sc.install_pypi_package("boto3==1.26.59")
sc.install_pypi_package("joblib==1.2.0")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from itertools import groupby
from operator import itemgetter

from lifelines import CoxPHFitter
from scipy.optimize import minimize

import pandas as pd
import boto3
import os
import pickle
import tempfile
import joblib
import io
import numpy as np
import matplotlib.pyplot as plt

MongoUrl = "mongodb+srv://xxxxxxxxxxxxxxxxxxxxxx/"

In [ ]:
spark = SparkSession \
    .builder \
    .appName("FinancialSheets_ML_Training") \
    .config("spark.cores.max", 6) \
    .config("spark.executor.cores", 6) \
    .config("spark.executor.memory", "36g") \
    .getOrCreate()

In [ ]:
reportRiskPremiumDf = spark.read.format("mongodb") \
    .option("spark.mongodb.read.connection.uri", MongoUrl) \
    .option("spark.mongodb.write.connection.uri", MongoUrl) \
    .option("database", "coreEngine") \
    .option("collection", "ReportRiskPremium_New") \
    .load() \
    .drop("_id")

In [ ]:
matchPipe = "{ $match: { updateDate: { $gte: '20200101' } } }"
    
priceDf = spark.read.format("mongodb") \
    .option("spark.mongodb.read.connection.uri", MongoUrl) \
    .option("spark.mongodb.write.connection.uri", MongoUrl) \
    .option("aggregation.pipeline", matchPipe) \
    .option("database", "coreEngine") \
    .option("collection", "Price") \
    .load() \
    .drop("_id") \
    .select("stockCode", "updateDate", "closingPrice","marketCap") \
    .withColumn('rolling', F.lag(F.col('closingPrice'), offset=90).over(Window.partitionBy('stockCode').orderBy(['updateDate']))) \
    .withColumn('closingPriceRolling', F.col('rolling') / F.col('closingPrice')) \
    .withColumn('r_s', F.when(F.col('closingPriceRolling') >= 1, 1 - 1).otherwise(F.col('closingPriceRolling') - 1)) \
    .withColumn('r', F.lit(0.07)) \
    .withColumn('recoveryFN1', F.lit(0.5)) \
    .withColumn('recoveryFN2', F.lit(0.1)) \
    .withColumn('recoveryFN3', F.lit(0.1)) \
    .withColumn('priceEvent', F.when(F.col('r_s') < -0.2, 1).otherwise(0)) \
    .withColumn('priceEvent', F.sum(F.col('priceEvent')).over(Window.partitionBy('stockCode').orderBy('updateDate').rowsBetween(-90, 0))) \
    .withColumn('priceEvent', F.when(F.col('priceEvent') > 0, 1).otherwise(0)) \
    .drop('closingPrice', 'closingPriceRolling', 'rolling') \
    .where(F.col("r_s").isNotNull())
    
    
    

In [ ]:
reportRiskPremiumDfPriceDf = reportRiskPremiumDf \
    .join(priceDf, on=["stockCode", "updateDate"], how="left") \
    .withColumn('TF1', F.sum(F.col('event')).over(Window.partitionBy('stockCode').orderBy(['updateDate'])) / F.count(F.col('event')).over(Window.partitionBy('stockCode').orderBy(['updateDate']))) \
    .withColumn('TF2', F.sum(F.col('priceEvent')).over(Window.partitionBy('stockCode').orderBy(['updateDate'])) / F.count(F.col('priceEvent')).over(Window.partitionBy('stockCode').orderBy(['updateDate']))) \
    .withColumn('TF3', F.sum(F.col('plbtEvent')).over(Window.partitionBy('stockCode').orderBy(['updateDate'])) / F.count(F.col('plbtEvent')).over(Window.partitionBy('stockCode').orderBy(['updateDate']))) \
    .withColumn("expectedProfit", F.lit(100000000 * 90/365) * F.col("r")) \
    .withColumn("expectedLossFN1", F.col("expectedProfit") * F.col("recoveryFN1") * F.col("TF1")) \
    .withColumn("expectedLossFN2", F.col("expectedProfit") * F.col("recoveryFN2") * F.col("TF2"))\
    .withColumn("expectedLossFN3", F.col("expectedProfit") * F.col("recoveryFN3") * F.col("TF3"))\

#     .withColumn('FSPctRank', F.percent_rank().over(Window.partitionBy("bsnsYear", "quarter").orderBy("riskPremium"))) \


In [ ]:
pdf = reportRiskPremiumDfPriceDf.toPandas()
periodDf = pdf[['updateDate']].drop_duplicates()

In [ ]:
date_list=sorted([p[0] for p in list(periodDf.values) if p[0] > '20210101'])

pdf_for_group = [] # 인스턴스가 누적으로 쌓이는 df의 리스트

'''
1. date_list = ['20220414', '20220427', '20220428', '20220502', ...] 
    =>  i보다 작은 것중 가장 큰 것을 가져옴으로 4월 14일보다 작은 값 중에서 가장 큰 값은 4월 11일임. (만약 4월 11일이 있다면)


<for문 과정>
    1번째
    i = 20220414
    updateDate <= 20220414. 애들 중에서 stockcode로 그룹화하여 각 기업별로 가장 최신의 것 하나.


    2번째 
    i = 20220427
    updateDate <= 20220427. 애들 중에서 stockcode로 그룹화하여 각 기업별로 가장 최신의 것 하나.
        =>따라서 1번째도 포함하여, 누적으로 df에 인스턴스가 쌓임

        
    마지막
    i == 20230814
    기업별 가장 최신의 값들만 있는 df가 됨.

'''

for i in date_list:
    temp_df = pdf[pdf['updateDate'] <= i].sort_values(by="updateDate", ascending=False).groupby("stockCode").head(1)
    temp_df["tempUpdateDate"]=i
    pdf_for_group.append(temp_df)

### predict  </br>
<pre>
    1. 과정을 통해 날짜별로 가장 최신의 df를 만듦.</br>
        - pdf_for_group 기준일에서 회사별 최신 정보로 업데이트된 df를 리스트로 묶음. 날짜별로 df가 리스트로 저장된 것.</br>
        - 각 df에서 riskPremium의 rank를 백분율로 표시 => FSPctRank 칼럼</br>
    2. get_threshold 함수</br>
        - 혼동행렬 수치로 얼마나 우리가 정의한 악재를 잘 구분하는지 계산.</br>
        - 최적화 통해 기준값 x를 산출 (minimize 함수 사용) => 긍정, 부정을 가장 잘 구분해내는 기준값 x
    3. predict 칼럼 생성
        - FSPctRank 가 x보다크면 1, 아니면 0 
        - 0이면 우량기업 일 확률 높음.
</pre>

In [ ]:
for df in pdf_for_group:
    df['FSPctRank'] = df['riskPremium'].transform('rank', pct=True) #순위가 아니라백분율로 표시

## 최적의 threshold x 찾기

* x보다 크면 => true => 1 > 악재 
* x보다 작으면 > Faslse > 0>호재

 최적화 함수 minize 함수를 metric이라는 변수에 저장. <br>
 그 결과값을 res에 저장하여 theshold를 찾는다.

## predict
> FSPctRank가 threshold보다 크면 True > ***predict = 1*** > 악재 <br>
> 작으면 False >  ***predict = 0***  > 호재


## 등급 기준
<pre>
    predict가 0인 기업들은 대출 가능 기업이고
        grade : A~D 
    predict가 1인 기업들은 악재발생한 기업이므로
        grade : F
    
    </pre>
## reGrade process
![a1](images/pngreGrade_rePredict.png)

In [ ]:
threshold_dfs = []

for i in pdf_for_group:
    
    def get_threshold(prob, df):
        df = df.fillna(0)    
        tn = (df['event'] == False).values * (prob == False).values # 현실 양성 | 예측 양성 
        fn1 = (df['event'] == True).values * (prob == False).values # 현실 악성 | 예측 양성 
        fn2 = (df['priceEvent'] == True).values * (prob == False).values # 수정주가 20 현실 악성 | 예측 양성   => 90일이 아니고?
        fn3 = (df['plbtEvent'] == True).values * (prob == False).values # 당기순이익 2분기 연속 악성 | 예측 양성
        tp = (df['event'] == True).values * (prob == True).values # 현실 악성 | 예측 악성
        fp = (df['event'] == False).values * (prob == True).values # 현실 양성 | 예측 악성

        x1 = df['expectedProfit'] @ tn
        x2 = df['expectedLossFN1'] @ fn1
        x3 = df['expectedLossFN2'] @ fn2
        x4 = df['expectedLossFN1'] @ tp
        x5 = df['expectedProfit'] @ fp

#         x6 = df['expectedLossFN3'] @ fn3
#         exret = (x1 - x2 - x4 + x5) / sum(df['expectedProfit'])
#         exret = (x1 - x2 - x3 - x4 - x6 + x5) / sum(df['expectedProfit'])
        exret = (x1 - x2 - x4 + x5) / sum(df['expectedProfit'])
        return exret
    

    tmp_df = i.reset_index(drop=True)
    median = np.median(tmp_df['FSPctRank'].values)
    print(median)
    metric = lambda x: get_threshold(prob=(tmp_df['FSPctRank'] > x), df=tmp_df)  #기준 x를 최적화함
    res = minimize(lambda x: -metric(x[0]), median, method='nelder-mead', options={'disp': False}) 
    threshold = res.x[0]
    tmp_df['threshold'] = threshold
    tmp_df['predict'] = (tmp_df['FSPctRank'] >= tmp_df['threshold']).astype(int)
    threshold_dfs.append(tmp_df)

threshold_dfs

'''
1. metric = lambda x: get_threshold(prob=(tmp_df['FSPctRank'] > x), df=tmp_df)
    metric이라는 것은 lambda로 정의된 함수임. 그래서 출력하면 lambda function으로 메모리에 저장되어 있음


2. minimize는 최적화 함수. 
    minimize(함수, 초기 추정값, method=  , options = )
    반환되는 값은 최적화된 변수를 반환. 변수가 여러개일 수 있으므로 리스트 형태로 반환됨=> x[0]인 이유 // 우리는 변수가 하나이므로 [x]의 형태임. 리스트 벗기려고 인덱스 사용하는 것

    <method 종류>
        'Nelder-Mead': Simplex algorithm (downhill simplex algorithm). 적은 차원의 문제에 유용합니다. 계산시간이 적거나 소규모인 문제, 노이즈가 있는 경우, 제한된 메모리 등에서 유용
        'Powell': Powell’s method. Conjugate direction method w0ith successive quadratic programming. 대칭 행렬을 다루는 데 효과적입니다.
        'CG': Conjugate Gradient algorithm. 특히 대칭, 양의 정부호 행렬에 대한 최적화에 효과적입니다.
        'BFGS': Broyden-Fletcher-Goldfarb-Shanno algorithm. 큰 규모의 최적화 문제에서 효과적입니다.
        'Newton-CG': Newton-Conjugate Gradient algorithm. Hessian matrix를 이용하여 적은 메모리를 사용하면서도 빠르게 최적화할 수 있습니다.
        'L-BFGS-B': Limited-memory Broyden-Fletcher-Goldfarb-Shanno with bounds. BFGS와 유사하나, 제한 조건이 있는 경우에 사용됩니다.
        'TNC': Truncated Newton-Conjugate Gradient algorithm. 제한된 최적화에 사용되며, 제한 조건이 있는 경우 효과적입니다.
        'COBYLA': Constrained Optimization BY Linear Approximations algorithm. 제한된 최적화에 사용되며, 제한 조건이 선형일 때 효과적입니다.
        'SLSQP': Sequential Least SQuares Programming. 비선형 최소화와 제한 조건이 있는 최적화에 사용됩니다.
        'trust-constr': Trust-region constrained algorithm. 제한된 최적화 문제를 다루며, 선형 또는 비선형 제한 조건을 포함할 수 있습니다.
        'trust-exact': Trust-region exact algorithm. 정확한 Hessian matrix를 이용하여 최적화합니다.
        'trust-krylov': Trust-region Krylov algorithm. 큰 규모의 비선형 제한 조건을 다룹니다.

    <option 종류>
        maxiter: 최대 반복 횟수를 지정합니다. 최적화 알고리즘이 이 횟수만큼 반복한 후 종료됩니다.
        disp: 최적화 알고리즘의 진행 상황을 표시할지 여부를 지정합니다. True로 설정하면 출력이 표시되고, False로 설정하면 출력이 표시되지 않습니다.
        ftol: 함수 값이 충분히 수렴되면 종료하는데 사용되는 상대적인 허용 오차입니다.
        gtol: 그래디언트의 크기가 충분히 작아지면 종료하는데 사용되는 상대적인 허용 오차입니다.
        xtol: 변수의 변화량이 충분히 작아지면 종료하는데 사용되는 상대적인 허용 오차입니다.
        maxfun: 최대 함수 호출 횟수를 지정합니다. 목적 함수의 평가가 이 횟수를 초과하면 최적화 알고리즘이 종료됩니다.
        maxfev: 최대 그래디언트 평가 횟수를 지정합니다.
        eps: 수치 그래디언트의 수치적 안정성을 제어하는 작은 양수입니다.

'''

In [ ]:
rank_dfs = []
import copy
for i in threshold_dfs:  
    tdf = i[~i.stockCode.isin(['012700','023460','334890','293940','396690','377190','365550','369370','400760','404990','064820','088260','350520','348950','395400','204210','145270'])]
#  제외 기업들은 리츠사임
# ~ : not 연산자
    loanAvailableDf = tdf[tdf['predict'] == 0]
    loanAvailableDf['loanFSPctRank'] = loanAvailableDf['riskPremium'].transform('rank', pct=True)
    loanAvailableDf['grade'] = 'D'
    loanAvailableDf.loc[(loanAvailableDf['loanFSPctRank'] <= 0.8, 'grade')] = 'C'
    loanAvailableDf.loc[(loanAvailableDf['loanFSPctRank'] <= 0.5, 'grade')] = 'B'
    loanAvailableDf.loc[(loanAvailableDf['loanFSPctRank'] <= 0.2, 'grade')] = 'A'
    loanAvailableDf.drop("loanFSPctRank", axis=1, inplace=True)
    loanNotAvailableDf = tdf[tdf['predict'] == 1]
    loanNotAvailableDf['grade'] = 'F'
    temp_df = pd.concat([loanAvailableDf, loanNotAvailableDf]).sort_values(by="FSPctRank", ascending=True).reset_index(drop=True)

    temp_grade = copy.deepcopy(temp_df['grade'].values)
    
    temp_df['rank'] = temp_df['FSPctRank'].rank(method='first')
    temp_df.loc[(temp_df['plbtEvent'] == 1), 'tmpRank'] = temp_df['rank'] + 50.5  # 최근 2분기 합계 영업이익이 <0 이하인 경우 가중치 

    temp_df.loc[(temp_df['plbtEvent'] == 0), 'tmpRank'] = temp_df['rank']
    temp_df['reRank'] = temp_df['tmpRank'].rank(method='first')
    temp_df = temp_df.sort_values(by="reRank", ascending=True)
    temp_df.drop("tmpRank", axis=1, inplace=True)
    temp_df['reGrade'] = temp_grade
    temp_df.loc[(temp_df['grade'] == 'F'), 'reGrade'] = 'F'
    
    temp_df.loc[(temp_df['reGrade'] == 'F'), 'rePredict'] = 1
    temp_df.loc[(temp_df['reGrade'] != 'F'), 'rePredict'] = 0
    
    rank_dfs.append(temp_df)


    '''
    rank(method = )  디폴트는 average
        ranking을 어떻게 나타내줄지를 결정하는 옵션입니다.
        min -> 동일한 rank일 경우 숫자가 더 작은(= 1위에 가까운) rank를 return
        max -> 동일한 rank일 경우 숫자가 더 큰(= 1위로부터 먼) rank를 return
        dense -> 동일한 rank 뒤에 나오는 rank를 바로 +1을 더한 rank로 표시함. (동일한 rank 다음 나오는 rank를 건너뛰고 표시하지 않음.)
        first -> 동일한 rank를 같은 순위로 표시하지 않고, 원본 data에서 더 위쪽에 있는 행에 숫자가 더 작은(= 1위에 가까운) rank를 부여함. 따라서 이 옵션은 rank에 따른 row number를 매길 수 있음.
        average -> first 옵션과 동일하지만 동일한 수치를 가져서 동일 rank인 행들에 대해 rank의 평균값을 부여함.
    '''

In [ ]:
dfs = pd.concat(rank_dfs).reset_index(drop=True)
sdf2 = spark.createDataFrame(dfs) \
        .withColumn("loanAvailable", F.col("predict")) \
        .withColumn("reLoanAvailable", F.col("rePredict")) \
        .withColumn("TT", F.lit(0.07)) \
        .withColumn("TT", F.when(F.col("grade") == "B", 0.075).otherwise(F.col("TT"))) \
        .withColumn("TT", F.when(F.col("grade") == "C", 0.085).otherwise(F.col("TT"))) \
        .withColumn("TT", F.when(F.col("grade") == "D", 0.095).otherwise(F.col("TT"))) \
        .withColumn("TT", F.when(F.col("grade") == "F", 0.1).otherwise(F.col("TT"))) \
        .withColumn("basicReturn", F.col("TT") * F.lit((90.0 / 365.0) * 100000000.0)) \
        .withColumn("basicReturn", F.when(F.col("grade") == "A", F.col("basicReturn").cast('Integer') * (F.lit(0.9) - F.col("FSPctRank"))).otherwise(F.col("basicReturn"))) \
        .withColumn("basicReturn", F.when(F.col("grade") == "B", F.col("basicReturn").cast('Integer') * (F.lit(0.8) - F.col("FSPctRank"))).otherwise(F.col("basicReturn"))) \
        .withColumn("basicReturn", F.when(F.col("grade") == "C", F.col("basicReturn").cast('Integer') * (F.lit(0.7) - F.col("FSPctRank"))).otherwise(F.col("basicReturn"))) \
        .withColumn("basicReturn", F.when(F.col("grade") == "D", F.col("basicReturn").cast('Integer') * (F.lit(0.5) - F.col("FSPctRank"))).otherwise(F.col("basicReturn"))) \
        .withColumn("basicReturn", F.when(F.col("grade") == "F", F.col("basicReturn").cast('Integer') * (F.lit(0.0) - F.col("FSPctRank"))).otherwise(F.col("basicReturn"))) \
        .withColumn("VaRTF1", F.col("TF1") * F.lit((90.0 / 365.0) * (100000000.0 * 0.5 * 0.08))) \
        .withColumn("VaRTF2", F.col("TF2") * F.lit((90.0 / 365.0) * (100000000.0 * 0.1 * 0.08))) \
        .withColumn("VaRTF3", F.col("TF3") * F.lit((90.0 / 365.0) * (100000000.0 * 0.1 * 0.08))) \
        .withColumn("expectedRisk", F.col("VaRTF1") + F.col("VaRTF2") + F.col("VaRTF3")) \
        .withColumn("profitLoss", F.col("basicReturn") - F.col("expectedRisk"))

In [ ]:
"""
sdf2 \
    .where(F.col("updateDate") > "20230530") \
    .write.format("mongodb") \
    .mode("append") \
    .option("upsertDocument", "true") \
    .option("idFieldList", "updateDate,tempUpdateDate,stockCode") \
    .option("spark.mongodb.read.connection.uri", MongoUrl) \
    .option("spark.mongodb.write.connection.uri", MongoUrl) \
    .option("database", "coreEngine") \
    .option("collection", "ReportRiskPremiumCalcurate_New") \
    .save()
    """

In [ ]:
spark.stop()